In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pydicom
import imageio
import os
from glob import glob
import nibabel as nib
from tqdm import tqdm
import json
import nibabel as nib
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import nibabel as nib
from glob import glob
from tqdm import tqdm
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

MRI

In [5]:
save_class_list=['Ischemic','Normal']
data_dir='../../data/raw_dicom/MRI/'
save_dir='../../data/stroke_mri_nii_256/images/'

In [6]:
target_shape = (64, 256, 256)  # (Z, H, W)

for j in range(len(save_class_list)):
    folder_list = glob(data_dir + save_class_list[j] + '/**')

    for i in tqdm(range(len(folder_list))):
        dicom_paths = glob(folder_list[i] + '/*.dcm')
        slices = []

        for dicom_file in sorted(dicom_paths):  # slice 순서 보존
            dcm = pydicom.dcmread(dicom_file)
            img = dcm.pixel_array.astype(np.float32)

            # RescaleSlope / Intercept 적용
            slope = float(getattr(dcm, 'RescaleSlope', 1))
            intercept = float(getattr(dcm, 'RescaleIntercept', 0))
            img = img * slope + intercept

            slices.append(img)

        volume = np.stack(slices, axis=0)  # (Z, H, W)
        patient_id = os.path.basename(folder_list[i]).replace('.dcm', '')

        # ✅ 정규화: -1 ~ 1
        vmin = np.percentile(volume, 5)
        vmax = np.percentile(volume, 100)

        volume = np.clip(volume, vmin, vmax)
        volume = (volume - vmin) / (vmax - vmin)  # → 0 ~ 1
        volume = volume * 2 - 1                   # → -1 ~ 1
        volume = volume.astype(np.float32)

        # ✅ 64x128x128로 리사이즈
        zoom_factors = [t / s for t, s in zip(target_shape, volume.shape)]
        volume_resized = zoom(volume, zoom_factors, order=1)  # trilinear interpolation

        # NIfTI 저장
        nifti_img = nib.Nifti1Image(volume_resized, affine=np.eye(4))
        save_path = os.path.join(save_dir, f"{patient_id}.nii.gz")
        create_dir(os.path.dirname(save_path))
        nib.save(nifti_img, save_path)

  0%|          | 0/752 [00:00<?, ?it/s]

100%|██████████| 361/361 [04:05<00:00,  1.47it/s]


In [3]:
import scipy.ndimage as ndi
def generate_largest_contour_mask_from_array(image):
    gray = image.astype(np.uint8)
    mask = np.zeros_like(gray, dtype=np.uint8)
    mask = np.ascontiguousarray(mask)

    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        cv2.fillPoly(mask, [largest_contour], 1)
    return mask
def generate_3d_foreground_mask(volume, threshold_percentile=70):
    # 1. threshold: 전체 volume에서 일정 percentile 이상만 foreground
    threshold = np.percentile(volume, threshold_percentile)
    binary = (volume > threshold).astype(np.uint8)

    # 2. connected component 분석 → 가장 큰 덩어리만 남김
    labeled, num_features = ndi.label(binary)
    sizes = ndi.sum(binary, labeled, range(1, num_features + 1))
    if len(sizes) == 0:
        return np.zeros_like(volume, dtype=np.uint8)
    largest_label = (np.argmax(sizes) + 1)
    mask = (labeled == largest_label).astype(np.uint8)

    return mask
save_class_list=['Ischemic','Normal']
data_dir='../../data/raw_dicom/MRI/'
save_dir='../../data/stroke_mri_nii_256/masks/'
mask_png_list=glob('../../data/stroke_mri/label/DWI/**')
mask_png_folder_list=[os.path.basename(path) for path in mask_png_list]
for j in range(len(save_class_list)):
    folder_list = glob(data_dir + save_class_list[j] + '/**')
    for i in tqdm(range(len(folder_list))):
        final_mask_volume = np.zeros((64, 256, 256), dtype=np.uint8)
        dcm_img=glob(folder_list[i]+'/*.dcm')
        dcm_img=sorted(dcm_img)
        slices = []
        for dicom_file in sorted(dcm_img):
            dcm = pydicom.dcmread(dicom_file)
            img = dcm.pixel_array.astype(np.float32)

            # RescaleSlope / Intercept 적용
            slope = float(getattr(dcm, 'RescaleSlope', 1))
            intercept = float(getattr(dcm, 'RescaleIntercept', 0))
            img = img * slope + intercept

            slices.append(img)
        volume = np.stack(slices, axis=0)  # (Z, H, W)
        patient_id = os.path.basename(folder_list[i]).replace('.dcm', '')

        # ✅ 정규화: -1 ~ 1
        vmin = np.percentile(volume,20)
        vmax = np.percentile(volume,90)

        volume = np.clip(volume, vmin, vmax)
        volume = (volume - vmin) / (vmax - vmin)  # → 0 ~ 1
        volume = volume*255                 # → -1 ~ 1
        volume = volume.astype(np.uint8)
        nii_data = volume

        folder_name=patient_id 
        if folder_name in mask_png_folder_list:
            tumor_label=True
            index=mask_png_folder_list.index(folder_name)
            tumor_mask_path=mask_png_list[index]
            tumor_mask_list=glob(tumor_mask_path+'/*.png')
        else:
            tumor_label=False
        mask1=generate_3d_foreground_mask(volume, threshold_percentile=60)
        for k in range(len(nii_data)):
            mask = mask1[k]
            mask=cv2.resize(mask, (128, 128), interpolation=cv2.INTER_NEAREST)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))
            mask = generate_largest_contour_mask_from_array(mask)

            if tumor_label:
                tumor_mask_list = sorted(tumor_mask_list)
                Valid_section1 = len(tumor_mask_list) - 64
                tumor_mask = cv2.imread(tumor_mask_list[Valid_section1 + k])
                tumor_mask = cv2.cvtColor(tumor_mask, cv2.COLOR_BGR2GRAY)
                tumor_mask = cv2.resize(tumor_mask, (256, 256))
                tumor_index = np.where(tumor_mask > 0)
                mask = cv2.resize(mask, (256, 256), interpolation=cv2.INTER_NEAREST)
                mask[tumor_index] = 2
            else:
                mask = cv2.resize(mask, (256, 256), interpolation=cv2.INTER_NEAREST)

            final_mask_volume[k] = mask
        create_dir(save_dir)
        # NIfTI로 저장
        affine = np.eye(4)  # 단순 저장용 단위 행렬, 필요 시 원본에서 복사도 가능
        nifti_img = nib.Nifti1Image(final_mask_volume, affine)
        nib.save(nifti_img, f"{save_dir}{folder_name}.nii.gz")

100%|██████████| 361/361 [03:17<00:00,  1.83it/s]


In [4]:
import scipy.ndimage as ndi
def generate_largest_contour_mask_from_array(image):
    gray = image.astype(np.uint8)
    mask = np.zeros_like(gray, dtype=np.uint8)
    mask = np.ascontiguousarray(mask)

    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        cv2.fillPoly(mask, [largest_contour], 1)
    return mask
def generate_3d_foreground_mask(volume, threshold_percentile=70):
    # 1. threshold: 전체 volume에서 일정 percentile 이상만 foreground
    threshold = np.percentile(volume, threshold_percentile)
    binary = (volume > threshold).astype(np.uint8)

    # 2. connected component 분석 → 가장 큰 덩어리만 남김
    labeled, num_features = ndi.label(binary)
    sizes = ndi.sum(binary, labeled, range(1, num_features + 1))
    if len(sizes) == 0:
        return np.zeros_like(volume, dtype=np.uint8)
    largest_label = (np.argmax(sizes) + 1)
    mask = (labeled == largest_label).astype(np.uint8)

    return mask
save_class_list=['Ischemic','Normal']
data_dir='../../data/raw_dicom/MRI/'
save_dir='../../data/stroke_mri_nii/masks/'
mask_png_list=glob('../../data/stroke_mri/label/DWI/**')
mask_png_folder_list=[os.path.basename(path) for path in mask_png_list]
for j in range(len(save_class_list)):
    folder_list = glob(data_dir + save_class_list[j] + '/**')
    for i in tqdm(range(len(folder_list))):
        final_mask_volume = np.zeros((64, 128, 128), dtype=np.uint8)
        dcm_img=glob(folder_list[i]+'/*.dcm')
        dcm_img=sorted(dcm_img)
        slices = []
        for dicom_file in sorted(dcm_img):
            dcm = pydicom.dcmread(dicom_file)
            img = dcm.pixel_array.astype(np.float32)

            # RescaleSlope / Intercept 적용
            slope = float(getattr(dcm, 'RescaleSlope', 1))
            intercept = float(getattr(dcm, 'RescaleIntercept', 0))
            img = img * slope + intercept

            slices.append(img)
        volume = np.stack(slices, axis=0)  # (Z, H, W)
        patient_id = os.path.basename(folder_list[i]).replace('.dcm', '')

        # ✅ 정규화: -1 ~ 1
        vmin = np.percentile(volume,20)
        vmax = np.percentile(volume,90)

        volume = np.clip(volume, vmin, vmax)
        volume = (volume - vmin) / (vmax - vmin)  # → 0 ~ 1
        volume = volume*255                 # → -1 ~ 1
        volume = volume.astype(np.uint8)
        nii_data = volume

        folder_name=patient_id 
        if folder_name in mask_png_folder_list:
            tumor_label=True
            index=mask_png_folder_list.index(folder_name)
            tumor_mask_path=mask_png_list[index]
            tumor_mask_list=glob(tumor_mask_path+'/*.png')
        else:
            tumor_label=False
        mask1=generate_3d_foreground_mask(volume, threshold_percentile=60)
        for k in range(len(nii_data)):
            mask = mask1[k]
            mask=cv2.resize(mask, (128, 128), interpolation=cv2.INTER_NEAREST)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))
            mask = generate_largest_contour_mask_from_array(mask)

            if tumor_label:
                tumor_mask_list = sorted(tumor_mask_list)
                Valid_section1 = len(tumor_mask_list) - 64
                tumor_mask = cv2.imread(tumor_mask_list[Valid_section1 + k])
                tumor_mask = cv2.cvtColor(tumor_mask, cv2.COLOR_BGR2GRAY)
                tumor_mask = cv2.resize(tumor_mask, (128, 128))
                tumor_index = np.where(tumor_mask > 0)
                mask = cv2.resize(mask, (128, 128), interpolation=cv2.INTER_NEAREST)
                mask[tumor_index] = 2
            else:
                mask = cv2.resize(mask, (128, 128), interpolation=cv2.INTER_NEAREST)

            final_mask_volume[k] = mask
        create_dir(save_dir)
        # NIfTI로 저장
        affine = np.eye(4)  # 단순 저장용 단위 행렬, 필요 시 원본에서 복사도 가능
        nifti_img = nib.Nifti1Image(final_mask_volume, affine)
        nib.save(nifti_img, f"{save_dir}{folder_name}.nii.gz")

100%|██████████| 361/361 [01:54<00:00,  3.14it/s]


CT


In [17]:
def process_image(gray):
    # 엣지 감지 및 팽창 (Canny + Dilate)
    edges = cv2.Canny(gray, 50, 150)
    dilated = cv2.dilate(edges, np.ones((3,3), np.uint8), iterations=2)

    # 컨투어 검출
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 기본 마스크 초기화
    mask_1 = np.zeros_like(gray, dtype=np.uint8)  # 큰 컨투어만 담을 마스크
    final_gray = gray.copy()                      # 최종 처리된 이미지
    mask_temp=np.zeros_like(gray, dtype=np.uint8)
    if len(contours) > 1:
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        largest_contour = contours[0]
        small_contours = contours[1:]

        # 작은 컨투어 마스크
        small_mask = np.zeros_like(gray, dtype=np.uint8)
        cv2.fillPoly(small_mask, small_contours, 255)
        final_gray[small_mask > 0] = 0  # 작은 컨투어 제거

        # 큰 컨투어만 채운 마스크
        cv2.fillPoly(mask_1, [largest_contour], 255)
        mask_temp[mask_1 > 0] = 255
    return final_gray, mask_temp
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [19]:
def window_and_normalize(img, dcm):
    # HU 변환
    slope = dcm.RescaleSlope if 'RescaleSlope' in dcm else 1
    intercept = dcm.RescaleIntercept if 'RescaleIntercept' in dcm else 0
    img = img * slope + intercept

    # WL/WW 적용
    wl = dcm.WindowCenter[0] if isinstance(dcm.WindowCenter, pydicom.multival.MultiValue) else dcm.WindowCenter
    ww = dcm.WindowWidth[0] if isinstance(dcm.WindowWidth, pydicom.multival.MultiValue) else dcm.WindowWidth

    if wl is not None and ww is not None:
        min_val = wl - ww / 2
        max_val = wl + ww / 2
        img = np.clip(img, min_val, max_val)
        img = (img - min_val) / (max_val - min_val)
    else:
        # fallback
        img = (img - np.min(img)) / (np.max(img) - np.min(img) + 1e-8)

    # [-1, 1] 스케일링
    img = img * 2 - 1
    return img

# ==========================
# ⚙️ 설정
# ==========================
class_list = ['Ischemic', 'Hemorrhagic', 'Normal']
save_path='../../data/stroke_ct_nii_256/'
for i in range(len(class_list)):
    slice_list = sorted(glob(f'../../data/raw_dicom/CT/{class_list[i]}/*'))

    for j in tqdm(range(len(slice_list))):
        dcm_list = sorted(glob(f'{slice_list[j]}/*.dcm'))
        nii_name = os.path.basename(slice_list[j])

        volume_img = []
        volume_mask = []

        for dcm_file in dcm_list:
            dcm = pydicom.dcmread(dcm_file)
            img = dcm.pixel_array
            img_norm = window_and_normalize(img, dcm)

            # 이미지 전처리 + 마스크 추출
            gray = (img_norm * 127.5 + 127.5).astype(np.uint8)
            img_proc, mask = process_image(gray)
            img_proc = cv2.resize(img_proc, (256, 256), interpolation=cv2.INTER_LINEAR)
            volume_img.append(img_proc.astype(np.float32)/127.5-1)
            mask = cv2.resize(mask, (256, 256), interpolation=cv2.INTER_NEAREST)
            volume_mask.append((mask > 0).astype(np.uint8))

        # Stack 3D
        volume_img = np.stack(volume_img, axis=0)   # shape: (D, H, W)
        volume_mask = np.stack(volume_mask, axis=0) # shape: (D, H, W)

        # Save as NIfTI
        nii_img = nib.Nifti1Image(volume_img, affine=np.eye(4))
        nii_mask = nib.Nifti1Image(volume_mask, affine=np.eye(4))

        os.makedirs(f'{save_path}images/{class_list[i]}', exist_ok=True)
        os.makedirs(f'{save_path}masks/{class_list[i]}', exist_ok=True)
        nib.save(nii_img, f'{save_path}images/{class_list[i]}/{nii_name}.nii.gz')
        nib.save(nii_mask, f'{save_path}masks/{class_list[i]}/{nii_name}.nii.gz')

  0%|          | 0/1010 [00:00<?, ?it/s]

100%|██████████| 474/474 [03:59<00:00,  1.98it/s]
